In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-38030?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for NIKE, Inc. Data & Machine Learning Engineer Undergraduate Internship

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsInternshipsNIKE, Inc. Data & Machine Learning Engineer Und

In [7]:
from langchain_core.prompts import PromptTemplate  

prompt_extract = PromptTemplate.from_template(  
    """  
    ### SCRAPED TEXT FROM WEBSITE:  
    {page_data}  
    ### INSTRUCTION:  
    The scraped text is from the career's page of a website.  
    Your job is to extract the job postings and return them in JSON format containing the following keys: `role`, `experience`, `skills` and `description`.  
    Only return the valid JSON.  
    ### VALID JSON (NO PREAMBLE).  
    """  
)  

chain_extract = prompt_extract | llm  
res = chain_extract.invoke(input={'page_data': page_data})  
type(res.content)

str

In [8]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'NIKE, Inc. Data & Machine Learning Engineer Undergraduate Internship',
 'experience': '0-2 years',
 'skills': ['Python',
  'SQL',
  'Tableau',
  'AI/ML techniques (neural networks, tree ensembles, regressions, etc.)',
  'Data structures and algorithms',
  'Solution and technical design',
  'Problem-solving, debugging, testing, and analytical skills'],
 'description': 'Apply machine learning and engineering expertise to improve existing algorithms and understand and be able to leverage core concepts of AI/ML to research and develop new algorithms. Collaborate in multi-functional teams to support other groups who will also be working on science or science-engineering-related work.'}

In [67]:
import pandas as pd

df = pd.read_csv("Portfolio.csv")

In [68]:
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular, .NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [69]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [72]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': ' https://example.com/ml-python-portfolio'},
  {'links': ' https://example.com/python-portfolio'}],
 [{'links': ' https://example.com/magento-portfolio'},
  {'links': ' https://example.com/wordpress-portfolio'}],
 [{'links': ' https://example.com/flutter-portfolio'},
  {'links': ' https://example.com/android-tv-portfolio'}],
 [{'links': ' https://example.com/ml-python-portfolio'},
  {'links': ' https://example.com/magento-portfolio'}],
 [{'links': ' https://example.com/magento-portfolio'},
  {'links': ' https://example.com/ml-python-portfolio'}],
 [{'links': ' https://example.com/typescript-frontend-portfolio'},
  {'links': ' https://example.com/full-stack-js-portfolio'}],
 [{'links': ' https://example.com/ml-python-portfolio'},
  {'links': ' https://example.com/python-portfolio'}]]

In [71]:
job = json_res
job['skills']

['Python',
 'SQL',
 'Tableau',
 'AI/ML techniques (neural networks, tree ensembles, regressions, etc.)',
 'Data structures and algorithms',
 'Solution and technical design',
 'Problem-solving, debugging, testing, and analytical skills']

In [74]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Ansh, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Data & Machine Learning Engineering Solutions for NIKE, Inc.

Dear Hiring Manager,

I came across the Data & Machine Learning Engineer Undergraduate Internship role at NIKE, Inc. and was impressed by the company's commitment to leveraging AI/ML techniques to drive innovation. As a Business Development Executive at AtliQ, I'd like to introduce our team's expertise in delivering tailored solutions that can support your goals.

At AtliQ, we specialize in AI & Software Consulting, empowering enterprises to scale, optimize processes, reduce costs, and enhance overall efficiency. Our team of experts has a proven track record in developing and implementing machine learning algorithms, data structures, and solution design. We're confident that our capabilities align with the requirements of the Data & Machine Learning Engineer role.

Our portfolio showcases our expertise in machine learning and Python development, which are essential skills for this role. You can explore our re